In [1]:
import gensim.models as gsm
from os import listdir
from os.path import isfile, join
from gensim.models.doc2vec import TaggedDocument
from collections import OrderedDict

In [2]:
#ruta a los archivos del corpus de entrada
train_corpus="./doc"

In [3]:
# etiquetar los archivos de texto
class DocIterator(object):
    def __init__(self, doc_list, labels_list):
        self.labels_list = labels_list
        self.doc_list = doc_list

    def __iter__(self):
        for idx, doc in enumerate(self.doc_list):
            yield TaggedDocument(words=doc.split(), tags=[self.labels_list[idx]])

docLabels = [f for f in listdir(train_corpus) if f.endswith('.txt')]
print(docLabels)
data = []
for doc in docLabels:
    data.append(open(join(train_corpus, doc), 'r').read())
    
it = DocIterator(data, docLabels)
#train doc2vec model
model = gsm.Doc2Vec(vector_size=300, window=10, min_count=1, workers=11,alpha=0.025, min_alpha=0.025) # use fixed learning rate
model.build_vocab(it)
model.train(it, total_examples=len(doc), epochs=20)


model.save("paper.model")

print("model is saved")

['texto1.txt', 'texto2.txt', 'texto3.txt', 'texto4.txt', 'texto5.txt', 'texto6.txt', 'texto7.txt', 'texto8.txt']
model is saved


In [4]:
# cargando el modelo
model="paper.model"
m=gsm.Doc2Vec.load(model)
print("model is loaded")

model is loaded


In [5]:
#ruta de acceso a los archivos de prueba
test_paper="./textos/S0210-48062009000300013-4.txt"
new_test = open(join(test_paper), 'r').read().split()
#print(new_test)
inferred_docvec = m.infer_vector(new_test)
m.docvecs.most_similar([inferred_docvec], topn=3)
#print('%s:\n %s' % (model, m.docvecs.most_similar(positive=[inferred_docvec], topn=5)))

[('texto6.txt', 0.9999697804450989),
 ('texto5.txt', 0.9999686479568481),
 ('texto4.txt', 0.9999674558639526)]